In [33]:
import numpy as np
import math
import matplotlib as mpl
from qiskit import QuantumCircuit, QuantumRegister
import qiskit.quantum_info as qi

In [79]:
from pytket.circuit import Circuit, DiagonalBox, QControlBox
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import CircBox

In [159]:
circ = Circuit(n+3)
diagonal_box = DiagonalBox(diagonal)
controlled_u_gate = QControlBox(diagonal_box, 1)
circ.add_gate(controlled_u_gate, [1,4,5,6])
render_circuit_jupyter(circ)

In [145]:
# prep_circuit = Circuit(1).X(0)
# unitary_circuit = Circuit(1).U1(input_angle, 0)

def create_diagonal_arr(n):
    N = int(math.pow(2, n))
    return [np.exp(2*math.pi*1j*theta/N) for theta in range(0, N)]

def create_prep_circuit(state: int, p: int):
    max_decimal = 2**p - 1
    min_binary_digits = max_decimal.bit_length()
    num_state = state
    if num_state > max_decimal:
        return None
    else:
        # Convert number to binary and ensure it has p digits
        binary = format(num_state, f"0{max(min_binary_digits, p)}b")
    prep_circ: Circuit = Circuit(p)
    binary = binary[::-1]
    for i in range(len(binary)):
        ch = binary[i]
        if ch == '1':
            prep_circ.X(i)
    return prep_circ

def build_qft_circuit(n_qubits: int) -> Circuit:
    circ = Circuit(n_qubits, name="QFT")
    for i in range(n_qubits):
        circ.H(i)
        for j in range(i + 1, n_qubits):
            circ.CU1(1 / 2 ** (j - i), j, i)
    for k in range(0, n_qubits // 2):
        circ.SWAP(k, n_qubits - k - 1)
        
    return circ

In [115]:
render_circuit_jupyter(prep_circ)

In [180]:
def create_textbook_QPE(
    m: int, p: int, state_prep_circuit: Circuit, diagonal) -> Circuit:
    # Define a Circuit with a measurement and prep register
    qpe_circ: Circuit = Circuit()
    #n_state_prep_qubits = state_prep_circuit.n_qubits
    measurement_register = qpe_circ.add_q_register("m", m)
    state_prep_register = qpe_circ.add_q_register("p", p)
    qpe_circ.add_circuit(state_prep_circuit, list(state_prep_register))

    # Create a controlled unitary with a single control qubit
    # unitary_circuit.name = "U"
    # controlled_u_gate = QControlBox(CircBox(unitary_circuit), 1)

    # Create a controlled unitary with a single control qubit
    circ = Circuit(n, name="U")
    diagonal_box = DiagonalBox(diagonal)
    controlled_u_gate = QControlBox(diagonal_box, 1)

    # Add Hadamard gates to every qubit in the measurement register
    for m_qubit in measurement_register:
        qpe_circ.H(m_qubit)

    # Add all (2**n_measurement_qubits - 1) of the controlled unitaries sequentially
    p_registers = [i + m for i in range(p)]
    for i in range(m):
        control_index = m - i - 1
        # print(control_index)
        # measure_i = measurement_register[control_index]
        control_qubit = [measurement_register[control_index]]
        combined = control_qubit + list(state_prep_register)
        #combined = [control_index]+p_registers
        print(combined)
        for _ in range(2**i):
            qpe_circ.add_gate(controlled_u_gate, combined)
            # qpe_circ.add_qcontrolbox(
            #     controlled_u_gate, control_qubit + list(state_prep_register)
            # )

    # Finally, append the inverse qft and measure the qubits
    qft_box = CircBox(build_qft_circuit(m))
    inverse_qft_box = qft_box.dagger
    qpe_circ.add_circbox(inverse_qft_box, list(measurement_register))
    qpe_circ.measure_register(measurement_register, "c")
    return qpe_circ

In [200]:
# DEFINE AND RUN CIRCUIT HERE
state = 3
p = 2
m = 4
prep_circ = create_prep_circuit(state, p)
diag = create_diagonal_arr(p)
circuit = create_textbook_QPE(m, p, prep_circ, diag)

[m[3], p[0], p[1]]
[m[2], p[0], p[1]]
[m[1], p[0], p[1]]
[m[0], p[0], p[1]]


In [201]:
render_circuit_jupyter(circuit)

In [192]:
from pytket.extensions.nexus import NexusBackend, QuantinuumConfig, Nexus
from pytket.passes import DecomposeBoxes

In [202]:
nexus = Nexus()
backend_name = "1E"
phase_est_project = Nexus().get_project_by_name(f"Standard QPR {backend_name}, state = {state}, p = {p}, m = {m}")
#phase_est_project = Nexus().new_project(f"Standard QPR {backend_name}, state = {state}, p = {p}, m = {m}")

configuration = QuantinuumConfig(device_name="H1-1E")

DecomposeBoxes().apply(circuit)

backend = NexusBackend(
    backend_config= configuration, 
    project= phase_est_project
)


Started using project with name: Standard QPR 1E, state = 3, p = 2, m = 7


In [203]:
compiled_circ = backend.get_compiled_circuit(circuit)

In [204]:
n_shots = 1
result = backend.run_circuit(compiled_circ, n_shots)

In [ ]:
print(result.get_counts())

In [ ]:
from pytket.backends.backendresult import BackendResult
import matplotlib.pyplot as plt

In [ ]:
def plot_qpe_results(
    sim_result: BackendResult,
    n_strings: int = m,
    dark_mode: bool = False,
    y_limit: int = 1000,
) -> None:
    """
    Plots results in a barchart given a BackendResult. the number of stings displayed
    can be specified with the n_strings argument.
    """
    counts_dict = sim_result.get_counts()
    sorted_shots = counts_dict.most_common()
    n_most_common_strings = sorted_shots[:n_strings]
    x_axis_values = [str(entry[0]) for entry in n_most_common_strings]  # basis states
    y_axis_values = [entry[1] for entry in n_most_common_strings]  # counts
    if dark_mode:
        plt.style.use("dark_background")
    fig = plt.figure()
    ax = fig.add_axes((0, 0, 0.75, 0.5))
    color_list = ["orange"] * (len(x_axis_values))
    ax.bar(
        x=x_axis_values,
        height=y_axis_values,
        color=color_list,
    )
    ax.set_title(label="Results")
    plt.ylim([0, y_limit])
    plt.xlabel("Basis State")
    plt.ylabel("Number of Shots")
    plt.show()

In [ ]:
plot_qpe_results(result, y_limit=int(1.2 * n_shots))

In [68]:
controlU.get_unitary()

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.

In [34]:
n = 2
N = int(math.pow(2, n))
matrix = []
for i in range(N):
    row = [0]*N
    theta = i + 1
    row[i] = math.cos(2*math.pi*theta/N) + 1j*math.sin(2*math.pi*theta/N)
    #math.exp(2*math.pi*1j/N) 
    matrix.append(row)
print(matrix)

[[(6.123233995736766e-17+1j), 0, 0, 0], [0, (-1+1.2246467991473532e-16j), 0, 0], [0, 0, (-1.8369701987210297e-16-1j), 0], [0, 0, 0, (1-2.4492935982947064e-16j)]]


In [38]:
custom_op = qi.Operator(matrix)

qr = QuantumRegister(n + 1)
qc = QuantumCircuit(qr)
qc.unitary(custom_op, [1,2], label="custom CX").control()
qc.draw("mpl")
# use DiagonalBox and ControlBox

AttributeError: 'InstructionSet' object has no attribute 'control'

In [14]:
qc.measure()

TypeError: QuantumCircuit.measure() missing 2 required positional arguments: 'qubit' and 'cbit'